In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 4007)


In [2]:
# Set a random seed to reproduce the results
np.random.seed(1234)

# Load the volume data
volume_data = pd.read_csv('../data/volume_data.csv', header=None)

In [3]:
all_hfs = pd.read_csv('../data/hf_list.csv')

used_hfs = [6336, 16572, 3393, 13632, 3592, 13829, 268, 6278, 10522, 16515, 16517,
            5539, 9615, 15773, 19854, 6782, 17022, 9424, 19659]

def find_index_hf(hf_no):
    return all_hfs[all_hfs['GID'] == hf_no].index.tolist()[0]

hfs = [find_index_hf(hf_no) for hf_no in used_hfs]

means = []
maxs = []

In [4]:
# Return a training and test data for a site.
def train_test_traffic_data(sequence_length=50):
    sample_size = volume_data.shape[0]
    result = []
    # scale data
    # Create data
    for j in hfs:
        road_vol = volume_data[j]
        road_vol = road_vol.replace(0, int(road_vol.mean())).values
        means.append(road_vol.mean())
        maxs.append(road_vol.max())
        road_vol = (road_vol - road_vol.mean())/road_vol.max()
        temp = []
        for i in range(0, sample_size - sequence_length):
            temp.append(road_vol[i: i + sequence_length])
        result.append(temp)

    result = np.dstack(result)
    row = round(0.9 * result.shape[0])
    train = result[:row, :, :]
    np.random.shuffle(train)
    X_train = train[:, :-1, :]
    y_train = train[:, -1, :]
    X_test = result[row:, :-1, :]
    y_test = result[row:, -1, :]

    return [X_train, y_train, X_test, y_test]

In [5]:
def build_model():
    mdl = Sequential()
    io_dim = len(hfs)
    # a network with 1-dimensional input,
    # two hidden layers of sizes 50, 100 and 100
    # and eventually a 1-dimensional output layer
    layers = [io_dim, 100, 200, 200, io_dim]

    # We also add 20% Dropout in this layer.
    mdl.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.2))
    
    # 2nd hidden layer
    mdl.add(LSTM(
        layers[2],
        return_sequences=True))
    mdl.add(Dropout(0.2))

    # 3rd hidden layer
    mdl.add(LSTM(
        layers[3],
        return_sequences=False))
    mdl.add(Dropout(0.2))

    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[4]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="rmsprop")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [6]:
def run_network(mdl=None, data=None):
    global_start_time = time.time()
    epochs = 20
    sequence_length = 50

    if data is None:
        print('Loading data... ')
        X_train, y_train, X_test, y_test = train_test_traffic_data(sequence_length)
    else:
        X_train, y_train, X_test, y_test = data

    print('\nData Loaded. Compiling...\n')

    if mdl is None:
        mdl = build_model()

    try:
        mdl.fit(X_train, y_train, batch_size=512,
                nb_epoch=epochs, validation_split=0.05)
        predicted_trffic = mdl.predict(X_test)
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return mdl, y_test, 0

    print('Training duration (s) : ', time.time() - global_start_time)

    return mdl, y_test, predicted_trffic

In [7]:
data = train_test_traffic_data(100)

In [8]:
model, y_test, predicted = run_network(data=data)


Data Loaded. Compiling...

Compilation Time :  0.21184515953063965
Train on 166782 samples, validate on 8779 samples
Epoch 1/20
166782/166782 [==============================] - 310s - loss: 0.0088 - val_loss: 0.0042
Epoch 2/20
166782/166782 [==============================] - 311s - loss: 0.0052 - val_loss: 0.0035
Epoch 3/20
166782/166782 [==============================] - 313s - loss: 0.0046 - val_loss: 0.0033
Epoch 4/20
166782/166782 [==============================] - 313s - loss: 0.0042 - val_loss: 0.0031
Epoch 5/20
166782/166782 [==============================] - 313s - loss: 0.0040 - val_loss: 0.0029
Epoch 6/20
166782/166782 [==============================] - 313s - loss: 0.0038 - val_loss: 0.0028
Epoch 7/20
166782/166782 [==============================] - 313s - loss: 0.0037 - val_loss: 0.0028
Epoch 8/20
166782/166782 [==============================] - 313s - loss: 0.0035 - val_loss: 0.0027
Epoch 9/20
166782/166782 [==============================] - 313s - loss: 0.0035 - val_loss

In [9]:
def plot_predictions(y_test, predicted):
    x = np.arange(400)
    y_test_tp = np.transpose(y_test)
    pred_tp = np.transpose(predicted)
    i = 13 #hf no 15773
    actual = y_test_tp[i][:400]
    predictions = pred_tp[i][:400]
    actual = (actual * maxs[i]) + means[i]
    predictions = (predictions * maxs[i]) + means[i]
    plt.plot(x,actual, label='Actual')
    plt.plot(x,predictions, label='Predicted')
    plt.legend(loc=2)
    plt.savefig('../latex-thesis/Figures/lstm-multi.pdf')
    plt.close()

In [10]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def print_scores(y_test, predicted):
    y_test_tp = np.transpose(y_test)
    pred_tp = np.transpose(predicted)
    mae = []
    mse = []
    mape = []
    #for i in range(len(hfs)):
    i=13
    actual = y_test_tp[i]
    predictions = pred_tp[i]
    actual = (actual * maxs[i]) + means[i]
    predictions = (predictions * maxs[i]) + means[i]
    mae.append(mean_absolute_error(actual, predictions))
    mse.append(mean_squared_error(actual, predictions))
    mape.append(mean_absolute_percentage_error(actual, predictions))
    print("MAE=", np.array(mae).mean())
    print("MSE=", np.array(mse).mean())
    print("MAPE=", np.array(mape).mean())

In [12]:
plot_predictions(y_test, predicted)

In [17]:
print_scores(y_test, predicted)

MAE= 15.1392187394
MSE= 479.762545996
MAPE= 17.7729902716
